In [1]:
import numpy as np
import pandas as pd
import tweepy
import os
from tweepy import OAuthHandler
import csv
import json

In [4]:
consumer_key = 'JBbZgOVsUDUbIyI3LLpS39b5A'
consumer_secret = '6jZ4JViRsK302RTsc6KDfBccMS4mI4fGjHzScxOCJA7PvWFvIo'
access_token = '568966944-T4wQcwC2Lm8fh2Vcgq02vY371RRlqDYqVOxx5sgE'
access_secret = '6K7PqTYXWNx0OJoHpNuFwbEOfuXiUoZEkAUCGsSzRyvhs'
 
auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)
 
api = tweepy.API(auth)

In [6]:
os.chdir('../Dataset')
parties_df = pd.read_csv('partyHandles.csv')
parties = parties_df['Party Handles'].values
#print(parties)
parties = [p.encode('ascii', 'ignore') for p in parties]
parties = [p.strip() for p in parties]
print(parties)

[b'BJP4Karnataka', b'BJP4India', b'INCKarnataka', b'INCIndia', b'INCTamilNadu', b'PMOIndia', b'cpimspeak', b'MahilaCongress', b'INCSandesh', b'NCPspeaks', b'AITCofficial', b'BJP4Maharashtra', b'BJP4Delhi', b'BJP4Gujarat', b'BJP4UP']


In [13]:
def get_user_tweets(screen_name, count = 200, from_id = None):
    print('Fetching tweets for user: {0}'.format(screen_name))
    all_tweets = []
    if not from_id:
        new_tweets = api.user_timeline(screen_name = screen_name, count = count)
    else:
        new_tweets = api.user_timeline(screen_name = screen_name, count = count, max_id = from_id)
        
    all_tweets.extend(new_tweets)
    oldest = all_tweets[-1].id - 1

    try:
        while len(new_tweets) > 0:
            new_tweets = api.user_timeline(screen_name = screen_name,count=200,max_id=oldest)
            #print('Added {0} new tweets'.format(len(new_tweets)))
            all_tweets.extend(new_tweets)
            oldest = all_tweets[-1].id - 1
    except Exception as e:
        print(str(e))
        
    print('Total tweets for {0}: {1}'.format(screen_name, len(all_tweets)))
    return all_tweets

def is_retweet(tweet):
    return 'RT @' in tweet.text

In [14]:
tweets = []
for p in parties:
    try:
        p_tweets = get_user_tweets(p, count = 1)
        tweets.extend(p_tweets)
    except Exception as e:
        print('Exception {0} while fetching tweets for {1}'.format(e, p))

Fetching tweets for user: b'BJP4Karnataka'
Total tweets for b'BJP4Karnataka': 3218
Fetching tweets for user: b'BJP4India'
Total tweets for b'BJP4India': 3246
Fetching tweets for user: b'INCKarnataka'
Total tweets for b'INCKarnataka': 3222
Fetching tweets for user: b'INCIndia'
Total tweets for b'INCIndia': 3229
Fetching tweets for user: b'INCTamilNadu'
Total tweets for b'INCTamilNadu': 3250
Fetching tweets for user: b'PMOIndia'
Total tweets for b'PMOIndia': 3220
Fetching tweets for user: b'cpimspeak'
Total tweets for b'cpimspeak': 3208
Fetching tweets for user: b'MahilaCongress'
Total tweets for b'MahilaCongress': 2926
Fetching tweets for user: b'INCSandesh'
Total tweets for b'INCSandesh': 219
Fetching tweets for user: b'NCPspeaks'
Total tweets for b'NCPspeaks': 3249
Fetching tweets for user: b'AITCofficial'
Total tweets for b'AITCofficial': 3207
Fetching tweets for user: b'BJP4Maharashtra'
Total tweets for b'BJP4Maharashtra': 3217
Fetching tweets for user: b'BJP4Delhi'
Total tweets for

In [15]:
with open('partyTweets.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerow(["Tweet_id", "User_Name", "User_Screen_Name", "Text"] )
    for tweet in tweets:
        if not is_retweet(tweet):
            writer.writerow([tweet.id_str, tweet.user.name, tweet.user.screen_name, tweet.text])

In [16]:
len(tweets)

45087